In [1]:
import numpy as np
import insertvalue.inter_nearest as iv 

#源文件路径
path='F:\\SCUT\\output2\\Couch_GB1_188\\'
#处理后 矩阵大小
maxlen=48

#digit标签，start end作者编号
def get_train_data(digit, start, end):
    x = np.ones([1, maxlen,maxlen])
    y = []   
    for i in range(start, end ):
        filepath = "{}GB1_%03d\\{}.txt".format(path, digit) % (i+1)
                
        #汉字处理
        label= digit

 #字母处理
#         labelint =int(digit[digit.index('_')+1:])+ 9  # A=10 B=11...a=36
#         if(labelint<=35) :
#             label=chr(int(labelint)+55)
#         else:
#             label=chr(int(labelint)+61)
           
 #数字处理   
#         if digit=='０':
#             label = "0"
#         elif digit== '１':
#             label = "1"
#         elif digit=='２':
#             label = "2"
#         elif digit=='３':
#             label = "3"
#         elif digit=='４':
#             label = "4"
#         elif digit=='５':
#             label = "5"
#         elif digit== '６':
#             label = "6"
#         elif digit=='７':
#             label = "7"
#         elif digit=='８':
#             label = "8"
#         elif digit=='９':
#             label = "9"
        


        a=np.loadtxt(filepath)
        #如果txt文件为空 ，读取前一个样本
        if(a.size== 0):
            a=np.loadtxt("{}GB1_%03d\\{}.txt".format(path, digit) % (i))
            
        #放缩、居中、补点       
        xymin=np.min(a,axis=0)
        xymax=np.max(a,axis=0)
        xmaxlen=xymax[1]-xymin[1]+1
        ymaxlen=xymax[0]-xymin[0]+1
        xymaxlen=max(xmaxlen,ymaxlen)
        xyminlen=min(xmaxlen,ymaxlen)       
        kong=(xymaxlen-xyminlen)/2
        flag=True
        if(xymaxlen==xmaxlen):flag=False
        file= np.ones([maxlen,maxlen])        
        if(xymaxlen>=maxlen):
            for i in range (0,len(a)):
                if(flag):
                    x1=int(a[i][1]-xymin[1]+kong)
                    x2=int(x1*(maxlen-1)/xymaxlen)
                    y1=int(a[i][0]-xymin[0])
                    y2=int(y1*(maxlen-1)/xymaxlen)
                else:
                    x1=int(a[i][1]-xymin[1])
                    x2=int(x1*(maxlen-1)/xymaxlen)
                    y1=int(a[i][0]-xymin[0]+kong)
                    y2=int(y1*(maxlen-1)/xymaxlen)
                if (x2>=maxlen or y2>=maxlen): continue
                file[x2][y2]=0
        else:
            pre_x=0
            pre_y=0
            for i in range (0,len(a)):
                #求 x2,y2
                if(flag):
                    x1=int(a[i][1]-xymin[1]+kong)
                    x2=int(x1*(maxlen-1)/xymaxlen)
                    y1=int(a[i][0]-xymin[0])
                    y2=int(y1*(maxlen-1)/xymaxlen)
                else:
                    x1=int(a[i][1]-xymin[1])
                    x2=int(x1*(maxlen-1)/xymaxlen)
                    y1=int(a[i][0]-xymin[0]+kong)
                    y2=int(y1*(maxlen-1)/xymaxlen)
                if (x2>=maxlen or y2>=maxlen): continue
                file[x2][y2]=0
                
                #补点
                cur_x=x2
                cur_y=y2
                if(i==0) :
                    pre_x=x2 
                    pre_y=y2
                    continue               
                if(abs(cur_x-pre_x)>5 or abs(cur_y-pre_y)>5 ):
                    pre_x=cur_x 
                    pre_y=cur_y
                    continue
                dx=cur_x-pre_x
                dy=cur_y-pre_y
                allpoint=[]
                if(abs(dy)>abs(dx)):
                    if(dx>0):
                        allpoint= drawlineP(pre_x,pre_y, cur_x,cur_y, "jumpEND",)
                    else:
                        allpoint= drawlineP(cur_x,cur_y, pre_x,pre_y, "jumpSTA",)
                else:
                    if(dy<0):
                        allpoint= drawlineP(cur_x,cur_y, pre_x,pre_y, "jumpSTA",)
                    else:
                        allpoint= drawlineP(pre_x,pre_y,cur_x,cur_y, "jumpEND",)   
                for i in range(0,len(allpoint)):
                    file[int(allpoint[i][0])][int(allpoint[i][1])]=0
                pre_x=cur_x
                pre_y=cur_y
        x = np.insert(x, 0, values=file, axis=0)       
        y.append(label)         
    y = np.array(y)
    x = np.delete(x, -1, axis = 0)
    return x, y

# 将处理后的矩阵保存

In [2]:
def save_data(digit, train_num, test_num):
    x_train, y_train = get_train_data(digit, 0, train_num)
    x_test, y_test = get_train_data(digit, train_num,train_num + test_num)
    np.savez('F:\SCUT\output2\GB1_48_48_center_in\\{}'.format(digit), x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

# 按标签处理

In [ ]:
train_num=160
test_num=35

#标签文件读取并保存
namepath="F:\\SCUT\\output2\\aname_gb1_10.txt" 
lines=open(namepath).readlines()
for data in lines:
    listname = data.strip('\n')
    save_data(listname,train_num,test_num)

# 补点方法



In [ ]:
def drawlineP(p0x,p0y,p1x,p1y,jump):   
    if(jump=="jumpSTA"):
        return drawlineNoStart(int(p0x), int(p0y), int(p1x), int(p1y))
    else:
        return drawlineNoEnd(int(p0x), int(p0y), int(p1x), int(p1y))

def drawlineNoStart(x0, y0, x1, y1):
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    sx, sy = 1, 1
    if(x0>=x1):
        sx=-1
    if(y0>=y1):
        sy=-1
    err=dx-dy
    nnn=[]
    while(True):
        e2=err*2
        if(e2<dx):
            err+=dx
            y0+=sy
        if(e2>-dy):
            err-=dy
            x0+=sx
        a=[x0,y0]
        nnn.append(a)
        if(x0==x1 and y0==y1):
            break
    return nnn
            

def drawlineNoEnd(x0, y0, x1, y1):
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    sx, sy = 1, 1
    if(x0>=x1):
        sx=-1
    if(y0>=y1):
        sy=-1
    err=dx-dy
    nnn=[]
    while(True):
        if(x0==x1 and y0==y1):
            break
        a=[x0,y0]
        nnn.append(a)
        e2=err*2
        if(e2<dx):
            err+=dx
            y0+=sy
        if(e2>-dy):
            err-=dy
            x0+=sx
    return nnn